In [1]:
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
datatrain=pd.read_csv('E:\\data\\fridaytrain.csv')

In [3]:
datatest=pd.read_csv('E:\\data\\blackfridaytest.csv')

In [4]:
na_percent=(datatrain.isnull().sum()/datatrain.isnull().count()).sort_values(ascending=False)

In [5]:
na_percent

Product_Category_3            0.696727
Product_Category_2            0.315666
Purchase                      0.000000
Product_Category_1            0.000000
Marital_Status                0.000000
Stay_In_Current_City_Years    0.000000
City_Category                 0.000000
Occupation                    0.000000
Age                           0.000000
Gender                        0.000000
Product_ID                    0.000000
User_ID                       0.000000
dtype: float64

In [6]:
#每件商品的均价
data_pm=datatrain.groupby('Product_ID')['Purchase'].mean().to_frame()
data_pm.columns=['Purchase_mean']
datatr=pd.merge(datatrain,data_pm,on='Product_ID')

In [7]:
#datatr表示包括均价和变换率的所有训练集信息
datatr['Purchase_rate']=(datatr.Purchase-datatr.Purchase_mean)/datatr.Purchase_mean

In [8]:
#target训练集的目标函数
target=datatr.Purchase_rate.to_frame()

In [9]:
 datatr.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Purchase_mean,Purchase_rate
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370,11870.863436,-0.294912
1,1000149,P00069042,M,26-35,1,B,2,1,3,NaN,NaN,10715,11870.863436,-0.097370
2,1000541,P00069042,F,18-25,4,C,3,0,3,NaN,NaN,11037,11870.863436,-0.070245
3,1000698,P00069042,M,18-25,4,A,1,0,3,NaN,NaN,8009,11870.863436,-0.325323
4,1000951,P00069042,M,46-50,2,B,4+,1,3,NaN,NaN,13346,11870.863436,0.124265


In [10]:
data0=datatr.drop(['Purchase','Purchase_rate'],axis=1)

In [11]:
data0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null object
Gender                        550068 non-null object
Age                           550068 non-null object
Occupation                    550068 non-null int64
City_Category                 550068 non-null object
Stay_In_Current_City_Years    550068 non-null object
Marital_Status                550068 non-null int64
Product_Category_1            550068 non-null int64
Product_Category_2            376430 non-null float64
Product_Category_3            166821 non-null float64
Purchase_mean                 550068 non-null float64
dtypes: float64(3), int64(4), object(5)
memory usage: 54.6+ MB


In [12]:
datatest0=pd.merge(datatest,data_pm,on='Product_ID')

In [13]:
datatest0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233538 entries, 0 to 233537
Data columns (total 12 columns):
User_ID                       233538 non-null int64
Product_ID                    233538 non-null object
Gender                        233538 non-null object
Age                           233538 non-null object
Occupation                    233538 non-null int64
City_Category                 233538 non-null object
Stay_In_Current_City_Years    233538 non-null object
Marital_Status                233538 non-null int64
Product_Category_1            233538 non-null int64
Product_Category_2            161236 non-null float64
Product_Category_3            71036 non-null float64
Purchase_mean                 233538 non-null float64
dtypes: float64(3), int64(4), object(5)
memory usage: 23.2+ MB


In [14]:
#按照id合并后平均价格没有空的，证明test数据集包含所有train数据集的product
datatest0.isnull().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2             72302
Product_Category_3            162502
Purchase_mean                      0
dtype: int64

In [15]:
data1=pd.concat([data0,datatest0],axis=0)

In [16]:
#产品类别进行独热编码，不能直接用get_dummies()
l=['Product_Category_1','Product_Category_2','Product_Category_3']
data2=data1[l]
data3=data1.drop(l,axis=1)

In [17]:
lcategory=list(data2.Product_Category_1.unique())

In [18]:
df1=pd.get_dummies(data2['Product_Category_1'])

In [19]:
df2=pd.get_dummies(data2['Product_Category_2'])

In [20]:
df3=pd.get_dummies(data2['Product_Category_3'])

In [21]:
ldf=list(df1.columns)

In [22]:
ldf

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [23]:
for c in ldf:
    if c not in list(df2):
        df2[c]=0
df2=df2.sort_index(axis=1)

In [24]:
for c in ldf:
    if c not in list(df3):
        df3[c]=0
df3=df3.sort_index(axis=1)

In [25]:
#至此用比较繁琐的方法完成了商品种类的独热处理
df=df1+df2+df3

In [26]:
def years2int(s):
    if s=='4+':
        s='5'
    s=int(s)
    return s

In [27]:
data3.Stay_In_Current_City_Years=data3.Stay_In_Current_City_Years.apply(years2int)

In [28]:
data3.Occupation=data3.Occupation.astype(str)

In [29]:
data4=data3.drop(['User_ID','Product_ID'],axis=1)

In [30]:
data4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783606 entries, 0 to 233537
Data columns (total 7 columns):
Gender                        783606 non-null object
Age                           783606 non-null object
Occupation                    783606 non-null object
City_Category                 783606 non-null object
Stay_In_Current_City_Years    783606 non-null int64
Marital_Status                783606 non-null int64
Purchase_mean                 783606 non-null float64
dtypes: float64(1), int64(2), object(4)
memory usage: 47.8+ MB


In [31]:
#data分为数值部分和数组部分
data_num=data4.select_dtypes(include='number')
data_str=data4.drop(data_num.columns,axis=1)

#数值部分归一化
from sklearn import preprocessing
mms=preprocessing.MinMaxScaler()
data_num.iloc[:,:]=mms.fit_transform(data_num.values)  

#字符串部分独热编码
data_str=pd.get_dummies(data_str)

#将3部分(data4的数值和字符部分，种类的单独独热编码部分df)合并成为新的data
data=pd.concat([data_num,data_str,df],axis=1)

D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [32]:
#将整个数据集用利于稀疏矩阵存储的方式scipy处理
import scipy 
from scipy import sparse
data_sparse = sparse.csc_matrix(data.values)

In [44]:
#划分训练集合测试集
train=data.iloc[:len(datatrain),:].values
test=data.iloc[len(datatrain):,:].values

In [40]:
#存储为稀疏矩阵形式
tr_sparse=sparse.csc_matrix(train.values)
ts_sparse=sparse.csc_matrix(test.values)

In [34]:
#对数据集进行stack 
#选取随机森林和xgboost两个方法

In [35]:
ytrain=target.values.reshape(-1,)

In [55]:
#xgboost
import xgboost as xgb
ps={}
ps['gamma']=[0.05,0.1,0.5,1]
ps['max_depth']=[6,8,10]
ps['learning_rate']=[0.05,0.1,0.3]
ps['objective']=['reg:linear']
ps['min_child_weight']=[0.5,1,5]
plst = list(ps.items())
num_rounds = 2000

#筛选最佳超参数
from sklearn.model_selection import GridSearchCV
xgbrg=xgb.XGBRegressor()
m1=GridSearchCV(xgbrg,ps)
#由于数据过多，gridsearch吃不消，选取部分数据来检查最佳参数
gindex=np.random.randint(len(train),size=5000)
sample=train[gindex,:]
ysample=ytrain[gindex]
m1.fit(sample,ysample)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'gamma': [0.05, 0.1, 0.5, 1], 'max_depth': [6, 8, 10], 'learning_rate': [0.05, 0.1, 0.3], 'objective': ['reg:linear'], 'min_child_weight': [0.5, 1, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [60]:
#用xgboost最优参数预测
xgbrg1=xgb.XGBRegressor(gamma=0.5,learning_rate=0.1,max_depth= 6,min_child_weight= 5,objective= 'reg:linear')
xgbrg1.fit(train,ytrain)
yxgb=xgbrg1.predict(train)

In [68]:
#存储xgboost模型
from sklearn.externals import joblib
joblib.dump(xgbrg1, 'xgbrg1.pkl')

['xgbrg1.pkl']

In [67]:
np.mean(abs(ytrain-yxgb))

0.2158536010351589

In [ ]:
# #randomforest
# ps2={}
# ps2['max_depth']=[5,7,9]
# ps2['n_estimators']=[100,500,800]

# from sklearn.ensemble import RandomForestRegressor
# rf=RandomForestRegressor()

# m2=GridSearchCV(rf,ps2)
# m2.fit(sample,ysample)


In [69]:
#用rf最优参数预测
rf1=xgb.XGBRegressor(max_depth= 6,n_estimators=500)
rf1.fit(train,ytrain)
yrf=xgbrg1.predict(train)
np.mean(abs(ytrain-yrf))

0.2158536010351589

In [70]:
yrf

array([ 0.00448534, -0.0115248 ,  0.0282408 , ...,  0.02126098,
       -0.01042801,  0.00192508], dtype=float32)

In [ ]:
from sklearn.externals import joblib
joblib.dump(rf1, 'rf1.pkl')

In [71]:
yxgb

array([ 0.00448534, -0.0115248 ,  0.0282408 , ...,  0.02126098,
       -0.01042801,  0.00192508], dtype=float32)

In [73]:
#将stack后的特征和原来的训练数据合并然后训练

# trainstack=np.vstack((yxgb,yrf)).T
trainstack=yxgb.reshape(-1,1)
tr=np.concatenate((train,trainstack),axis=1)
#训练，采用xgboost训练集成模型，参数用m1中表现最好的参数
xgbrg2=xgb.XGBRegressor(gamma=0.5,learning_rate=0.1,max_depth= 6,min_child_weight= 5,objective= 'reg:linear')
xgbrg2.fit(tr,ytrain)
from sklearn.externals import joblib
joblib.dump(xgbrg2, 'xgbrg2.pkl')

['xgbrg2.pkl']

In [75]:
#测试集计算
yxgb_ts=xgbrg1.predict(test)
yxrf_ts=rf1.predict(test)

# teststack=np.vstack((yxgb_ts,yrf_ts)).T
teststack=yxgb_ts.reshape(-1,1)
ts=np.concatenate((test,teststack),axis=1)
#测试
ytest=xgbrg2.predict(ts)


In [77]:
test0=datatest0[['User_ID','Product_ID']]
test0=pd.merge(test0,data_pm,on='Product_ID')
test0.loc[:,'Purchase_rate']=ytest
test0['Purchase']=test0['Purchase_rate']*test0['Purchase_mean']+test0['Purchase_mean']
test0.to_csv('E:\\data\\sb1.csv')